In [ ]:
# Hack to make the module importable
import sys
sys.path.append(r'./../')

In [ ]:
%load_ext autoreload
%autoreload 2
from py2neo import Graph, NodeMatcher
import pandas as pd

from rel2graph.relational_modules.pandas import PandasDataframeIterator
from rel2graph import IteratorIterator
from rel2graph import Converter
from rel2graph.utils import load_file
from rel2graph import register_attribute_postprocessor, Attribute
import rel2graph.common_modules

filename = "./pd_dataframe_demo_config.yaml"

In [ ]:
# Configure Logging
import logging

#logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger("rel2graph")
logger.setLevel(logging.DEBUG)
log_formatter = logging.Formatter("%(asctime)s [%(threadName)s]::[%(levelname)s]::%(filename)s: %(message)s")
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)


In [ ]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris

In [ ]:
data = {
  "ID": [1,2,2,3,4,4],
  "FirstName": ["Julian", "Fritz",  "Fritz", "Hans", "Rudolfo", "Rudolfo"],
  "LastName": ["Minder", "Generic", "SomeGuy", "Müller", "Muster", "Muster"],
  "FavoriteFlower": ["virginica", "setosa", "setosa", "versicolor", "setosa", "setosa"]
}
people = pd.DataFrame(data)
people

In [ ]:
graph = Graph(scheme="http", host="localhost", port=7474,  auth=('neo4j', 'password')) ## UPDATE password

graph.delete_all()  # reset graph (only wehn first creating the databse, here for debugging purposes)

In [ ]:
# Now neo4j does not support the numpy dtype int64, so we need to convert it to python native int
# We create a wrapper for this.
@register_attribute_postprocessor
def INT(attribute):
    return Attribute(attribute.key, int(attribute.value))

# In the schema file wrap the Person.ID attribute in the INT wrapper
#        + ID = INT(Person.ID)

In [ ]:
iterator = IteratorIterator([PandasDataframeIterator(people, "Person"), PandasDataframeIterator(iris, "Flower")])

In [ ]:
graph.delete_all()

In [ ]:
converter = Converter(load_file(filename), iterator, graph, num_workers=1)

In [ ]:
from tqdm.notebook import tqdm
converter(progress_bar=tqdm)